# Загружаем данные

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import DataCollatorForTokenClassification
from transformers import TrainingArguments, Trainer
from datasets import Dataset
import evaluate
from corus import load_ne5

In [2]:
dir = '../data/Collection5/'
records = load_ne5(dir)

In [3]:
df = pd.DataFrame(records)

In [4]:
df.rename(columns={0: 'id', 1: 'text', 2: 'ner'}, inplace=True)

In [5]:
df = df[['text', 'ner']]

In [6]:
df.head(4)

,text,ner
0,Жириновский предлагает обменять с США Сноудена...,"[Ne5Span(index='T1', type='PER', start=0, stop..."
1,Д.Медведев назначил ряд глав региональных МВД\...,"[Ne5Span(index='T1', type='PER', start=0, stop..."
2,СМИ: В.Суркову надоело работать в администраци...,"[Ne5Span(index='T1', type='MEDIA', start=0, st..."
3,Д.Медведев освободил от должности еще 10 генер...,"[Ne5Span(index='T1', type='PER', start=0, stop..."


# train test split и подготовка данных

In [7]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [8]:
# Получим все уникальные типы именованных сущностей
entity_types = set()
for spans in df['ner']:
    for span in spans:
        entity_types.add(span.type)

# Создадим маппинг для меток
id2label = {0: "O"}  # O - для токенов не входящих в именованную сущность
for i, entity_type in enumerate(sorted(entity_types), 1):
    id2label[2*i-1] = f"B-{entity_type}"  # B- префикс для начала сущности
    id2label[2*i] = f"I-{entity_type}"    # I- префикс для продолжения сущности
    
label2id = {v: k for k, v in id2label.items()}

print(f"Всего {len(entity_types)} типов сущностей: {sorted(entity_types)}")
print(f"Всего {len(id2label)} меток: {id2label}")

Всего 5 типов сущностей: ['GEOPOLIT', 'LOC', 'MEDIA', 'ORG', 'PER']
Всего 11 меток: {0: 'O', 1: 'B-GEOPOLIT', 2: 'I-GEOPOLIT', 3: 'B-LOC', 4: 'I-LOC', 5: 'B-MEDIA', 6: 'I-MEDIA', 7: 'B-ORG', 8: 'I-ORG', 9: 'B-PER', 10: 'I-PER'}


In [9]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")

In [10]:
# Функция для преобразования текста и NER-разметки в формат для обучения
def preprocess_data(examples):
    # Получаем текст и аннотации
    text = examples["text"]
    spans = examples["ner"]
    
    # Создаем список меток для каждого символа в тексте (по умолчанию 'O')
    char_labels = ["O"] * len(text)
    
    # Заполняем метки для сущностей
    for span in spans:
        entity_type = span.type
        start, end = span.start, span.stop
        
        # Устанавливаем B- для первого символа
        char_labels[start] = f"B-{entity_type}"
        
        # Устанавливаем I- для остальных символов в сущности
        for i in range(start + 1, end):
            char_labels[i] = f"I-{entity_type}"
    
    # Токенизируем текст
    tokenized = tokenizer(text, truncation=True, return_offsets_mapping=True)
    labels = []
    
    # Преобразуем символьные метки в метки токенов
    for i, (start, end) in enumerate(tokenized.offset_mapping):
        # Пропуск специальных токенов ([CLS], [SEP], ...)
        if start == end == 0:
            labels.append(-100)  # -100 игнорируется при вычислении потерь
            continue
            
        # Находим наиболее частую метку для символов в этом токене
        token_labels = char_labels[start:end]
        if not token_labels:
            labels.append(-100)
            continue
            
        # Если есть хотя бы одна B- метка, то используем её
        b_labels = [l for l in token_labels if l.startswith("B-")]
        if b_labels:
            labels.append(label2id[b_labels[0]])
        else:
            # Если есть хотя бы одна I- метка, то используем первую I- метку
            i_labels = [l for l in token_labels if l.startswith("I-")]
            if i_labels:
                labels.append(label2id[i_labels[0]])
            else:
                # Иначе используем "O"
                labels.append(label2id["O"])
    
    # Удаляем offset_mapping, так как он не нужен для обучения
    tokenized.pop("offset_mapping")
    
    tokenized["labels"] = labels
    return tokenized

In [11]:
label2id

{'O': 0,
 'B-GEOPOLIT': 1,
 'I-GEOPOLIT': 2,
 'B-LOC': 3,
 'I-LOC': 4,
 'B-MEDIA': 5,
 'I-MEDIA': 6,
 'B-ORG': 7,
 'I-ORG': 8,
 'B-PER': 9,
 'I-PER': 10}

In [12]:
# Применяем функцию к обучающей и тестовой выборкам
processed_train_data = []
processed_test_data = []

for i, row in train_df.iterrows():
    processed_train_data.append(preprocess_data(row))
    
for i, row in test_df.iterrows():
    processed_test_data.append(preprocess_data(row))

# Создаем датасеты Hugging Face
train_dataset = Dataset.from_dict({
    "input_ids": [x["input_ids"] for x in processed_train_data],
    "attention_mask": [x["attention_mask"] for x in processed_train_data],
    "labels": [x["labels"] for x in processed_train_data]
})

test_dataset = Dataset.from_dict({
    "input_ids": [x["input_ids"] for x in processed_test_data],
    "attention_mask": [x["attention_mask"] for x in processed_test_data],
    "labels": [x["labels"] for x in processed_test_data]
})

# дообучение модели

In [13]:
# Загрузка предобученной модели
model = AutoModelForTokenClassification.from_pretrained(
    "cointegrated/rubert-tiny2", 
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
# Функция для вычисления метрик
def compute_metrics(eval_preds):
    metric = evaluate.load("seqeval")
    
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    
    # Убираем игнорируемые индексы
    true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [19]:
# Оценка метрик без дообучения
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir='../data/models',
    per_device_eval_batch_size=8,
    no_cuda=not torch.cuda.is_available()
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


print("Метрики до дообучения:")
pre_training_metrics = trainer.evaluate(test_dataset)
pre_training_metrics

Метрики до дообучения:


/Users/kulyaskin_mikhail/Library/Caches/pypoetry/virtualenvs/dl-nlp-7PEEyLUf-py3.12/lib/python3.12/site-packages/transformers/training_args.py:1595: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


{'eval_loss': 2.3831734657287598,
 'eval_model_preparation_time': 0.0004,
 'eval_precision': 0.006539280014186913,
 'eval_recall': 0.057071000193461015,
 'eval_f1': 0.01173405461307452,
 'eval_accuracy': 0.12865108868826342,
 'eval_runtime': 4.9607,
 'eval_samples_per_second': 40.317,
 'eval_steps_per_second': 5.04}

In [21]:
training_args = TrainingArguments(
    output_dir='../data/models/rubert-tiny2-ner',
    eval_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    no_cuda=not torch.cuda.is_available()
)

/Users/kulyaskin_mikhail/Library/Caches/pypoetry/virtualenvs/dl-nlp-7PEEyLUf-py3.12/lib/python3.12/site-packages/transformers/training_args.py:1595: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

/var/folders/gb/r8_rgfms4txcy1_nl4l107kc0000gn/T/ipykernel_28611/2943834762.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.588897,0.465635,0.250339,0.325617,0.843717
2,No log,0.423745,0.441952,0.369704,0.402612,0.868262
3,No log,0.386830,0.472085,0.444960,0.458122,0.885422


/Users/kulyaskin_mikhail/Library/Caches/pypoetry/virtualenvs/dl-nlp-7PEEyLUf-py3.12/lib/python3.12/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kulyaskin_mikhail/Library/Caches/pypoetry/virtualenvs/dl-nlp-7PEEyLUf-py3.12/lib/python3.12/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kulyaskin_mikhail/Library/Caches/pypoetry/virtualenvs/dl-nlp-7PEEyLUf-py3.12/lib/python3.12/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. U

TrainOutput(global_step=300, training_loss=0.6780633036295572, metrics={'train_runtime': 488.9889, 'train_samples_per_second': 4.908, 'train_steps_per_second': 0.614, 'total_flos': 22374206344752.0, 'train_loss': 0.6780633036295572, 'epoch': 3.0})

In [25]:
# Оценка метрик после дообучения
post_training_metrics = trainer.evaluate(test_dataset)
post_training_metrics

{'eval_loss': 0.3868299722671509,
 'eval_precision': 0.4720853858784893,
 'eval_recall': 0.444960340491391,
 'eval_f1': 0.45812170102579425,
 'eval_accuracy': 0.8854221986192247,
 'eval_runtime': 4.3761,
 'eval_samples_per_second': 45.703,
 'eval_steps_per_second': 5.713,
 'epoch': 3.0}

#   MLM дообучение 

In [67]:
block_size = 128


def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [68]:
def preprocess_function(examples):
    return tokenizer(examples["text"])

In [69]:
# Применяем функцию к обучающей и тестовой выборкам
processed_train_data_mlm = []
processed_test_data_mlm = []

for i, row in train_df.iterrows():
    processed_train_data_mlm.append(preprocess_function(row))
    
for i, row in test_df.iterrows():
    processed_test_data_mlm.append(preprocess_function(row))

# Создаем датасеты Hugging Face
train_dataset_mlm = Dataset.from_dict({
    "input_ids": [x["input_ids"] for x in processed_train_data_mlm],
    "attention_mask": [x["attention_mask"] for x in processed_train_data_mlm],
})

test_dataset_mlm = Dataset.from_dict({
    "input_ids": [x["input_ids"] for x in processed_test_data_mlm],
    "attention_mask": [x["attention_mask"] for x in processed_test_data_mlm],
})

Token indices sequence length is longer than the specified maximum sequence length for this model (2308 > 2048). Running this sequence through the model will result in indexing errors


In [70]:
train_dataset_mlm_grouped = train_dataset_mlm.map(group_texts, batched=True, num_proc=4)
test_dataset_mlm_grouped = test_dataset_mlm.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/800 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/200 [00:00<?, ? examples/s]

In [76]:
from transformers import DataCollatorForLanguageModeling

tokenizer.add_special_tokens({'pad_token': '[PAD]'})
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [77]:
from transformers import AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained(
    "cointegrated/rubert-tiny2", 
)

In [78]:
training_args = TrainingArguments(
    output_dir="../data/models/rubert-tiny2-mlm",
    eval_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    no_cuda=not torch.cuda.is_available()
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_mlm_grouped,
    eval_dataset=test_dataset_mlm_grouped,
    data_collator=data_collator,
)

trainer.train()

/Users/kulyaskin_mikhail/Library/Caches/pypoetry/virtualenvs/dl-nlp-7PEEyLUf-py3.12/lib/python3.12/site-packages/transformers/training_args.py:1595: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,3.191179
2,No log,3.124142
3,No log,3.100579


There were missing keys in the checkpoint model loaded: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias'].


TrainOutput(global_step=372, training_loss=3.4454778855846775, metrics={'train_runtime': 317.2932, 'train_samples_per_second': 18.683, 'train_steps_per_second': 1.172, 'total_flos': 11310209703936.0, 'train_loss': 3.4454778855846775, 'epoch': 3.0})

In [79]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 23.54


# NER дообучение 

In [81]:
model = AutoModelForTokenClassification.from_pretrained(
    '../data/models/rubert-tiny2-mlm/checkpoint-372', 
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at ../data/models/rubert-tiny2-mlm/checkpoint-372 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [85]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")

In [86]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [87]:
training_args = TrainingArguments(
    output_dir='../data/models/rubert-tiny2-ner-v2',
    eval_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    no_cuda=not torch.cuda.is_available()
)

In [88]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

/var/folders/gb/r8_rgfms4txcy1_nl4l107kc0000gn/T/ipykernel_28611/2943834762.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.576352,0.459727,0.260592,0.332634,0.843949
2,No log,0.413024,0.442426,0.390985,0.415118,0.876958
3,No log,0.375955,0.491532,0.471658,0.481390,0.895031


/Users/kulyaskin_mikhail/Library/Caches/pypoetry/virtualenvs/dl-nlp-7PEEyLUf-py3.12/lib/python3.12/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kulyaskin_mikhail/Library/Caches/pypoetry/virtualenvs/dl-nlp-7PEEyLUf-py3.12/lib/python3.12/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kulyaskin_mikhail/Library/Caches/pypoetry/virtualenvs/dl-nlp-7PEEyLUf-py3.12/lib/python3.12/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. U

TrainOutput(global_step=300, training_loss=0.6769447835286458, metrics={'train_runtime': 487.8083, 'train_samples_per_second': 4.92, 'train_steps_per_second': 0.615, 'total_flos': 22374206344752.0, 'train_loss': 0.6769447835286458, 'epoch': 3.0})

- сравнительно с простым дообучением на ner задачу, получили метрики выше на каждой эпохе обучения

# обучение с доп разметкой

In [21]:
lenta_10000_labeled = pd.read_csv('../data/lenta_10000_labeled.csv')

In [22]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")

In [23]:
def prepare_ner_data(text, entities):
    # Получаем токены из текста
    encoding = tokenizer(text, return_offsets_mapping=True, padding=False, truncation=True)
    input_ids = encoding["input_ids"]
    offsets = encoding["offset_mapping"]
    
    # Инициализируем метки для каждого токена как "O" (outside)
    labels = ["O"] * len(input_ids)
    
    # Создаем словарь для маппинга позиции в тексте к индексу токена
    position_to_token_idx = {}
    for i, (start, end) in enumerate(offsets):
        if start != end:  # Пропускаем спецтокены с нулевой длиной
            for pos in range(start, end):
                position_to_token_idx[pos] = i
    
    # Размечаем токены на основе данных entities
    for entity in entities:
        # Получаем позиции в тексте
        start_pos = entity['start']
        end_pos = entity['end']
        entity_label = entity['entity']
        
        # Находим соответствующие токены
        token_indices = []
        for pos in range(start_pos, end_pos):
            if pos in position_to_token_idx:
                token_idx = position_to_token_idx[pos]
                if token_idx not in token_indices:
                    token_indices.append(token_idx)
        
        # Если найдены токены, применяем метки
        if token_indices:
            # Получаем тип сущности из метки
            if '-' in entity_label:
                _, entity_type = entity_label.split('-', 1)
            else:
                continue  # Пропускаем, если метка не в формате X-TYPE
            
            # Применяем метки в зависимости от кол-ва токенов
            for i, token_idx in enumerate(token_indices):
                if len(token_indices) == 1:
                    labels[token_idx] = f"U-{entity_type}"
                elif i == 0:
                    labels[token_idx] = f"B-{entity_type}"
                elif i == len(token_indices) - 1:
                    labels[token_idx] = f"L-{entity_type}"
                else:
                    labels[token_idx] = f"I-{entity_type}"
    
    return {
        "input_ids": input_ids,
        "attention_mask": [1] * len(input_ids),
        "labels": labels
    }

In [24]:
import ast

# Функция для конвертации строкового представления списка словарей в список словарей
def parse_entities(entities_str):
    if isinstance(entities_str, str):
        return ast.literal_eval(entities_str)
    return entities_str

In [25]:
lenta_train_data = []

In [26]:
# Обработка каждой строки датасета
for idx, row in lenta_10000_labeled.iterrows():
    try:
        text_data = prepare_ner_data(row['text'], parse_entities(row['text_entities']))        
        lenta_train_data.append(text_data)
            
        # Отображение прогресса
        if idx % 100 == 0:
            print(f"Обработано {idx} строк из {len(lenta_10000_labeled)}")
            
    except Exception as e:
        print(f"Ошибка при обработке строки {idx}: {e}")
        continue

Обработано 0 строк из 10000
Обработано 100 строк из 10000
Обработано 200 строк из 10000
Обработано 300 строк из 10000
Обработано 400 строк из 10000
Обработано 500 строк из 10000
Обработано 600 строк из 10000
Обработано 700 строк из 10000
Обработано 800 строк из 10000
Обработано 900 строк из 10000
Обработано 1000 строк из 10000
Обработано 1100 строк из 10000
Обработано 1200 строк из 10000
Обработано 1300 строк из 10000
Обработано 1400 строк из 10000
Обработано 1500 строк из 10000
Обработано 1600 строк из 10000
Обработано 1700 строк из 10000
Обработано 1800 строк из 10000
Обработано 1900 строк из 10000
Обработано 2000 строк из 10000
Обработано 2100 строк из 10000
Обработано 2200 строк из 10000
Обработано 2300 строк из 10000
Обработано 2400 строк из 10000
Обработано 2500 строк из 10000
Обработано 2600 строк из 10000
Обработано 2700 строк из 10000
Обработано 2800 строк из 10000
Обработано 2900 строк из 10000
Обработано 3000 строк из 10000
Обработано 3100 строк из 10000
Обработано 3200 стро

In [27]:
from datasets import Dataset

lenta_train_dataset = Dataset.from_dict({
    'input_ids': [x['input_ids'] for x in lenta_train_data],
    'attention_mask': [x['attention_mask'] for x in lenta_train_data],
    'labels': [x['labels'] for x in lenta_train_data]
})

In [28]:
def convert_lenta_labels_to_train_format(example):
    # Маппинг типов сущностей из lenta в типы train
    entity_type_mapping = {
        # Географические объекты -> LOC
        'CITY': 'LOC',
        'COUNTRY': 'LOC',
        'DISTRICT': 'LOC',
        'REGION': 'LOC',
        'STREET': 'LOC',
        'HOUSE': 'LOC',  # Дома также относим к локациям
        
        # Персоны -> PER
        'FIRST_NAME': 'PER',
        'LAST_NAME': 'PER',
        'MIDDLE_NAME': 'PER',
        
        # Другие типы можно добавить по мере необходимости
    }
    
    # Словарь меток train_dataset
    train_label2id = {
        'O': 0, 
        'B-GEOPOLIT': 1, 'I-GEOPOLIT': 2, 
        'B-LOC': 3, 'I-LOC': 4, 
        'B-MEDIA': 5, 'I-MEDIA': 6, 
        'B-ORG': 7, 'I-ORG': 8, 
        'B-PER': 9, 'I-PER': 10
    }
    
    # Преобразуем метки
    new_labels = []
    for label in example['labels']:
        if label == 'O':
            new_labels.append(train_label2id['O'])
            continue
            
        # Разбиваем метку на позицию (B/I/L/U) и тип сущности
        position, entity_type = label.split('-', 1)
        
        # Преобразуем тип сущности
        mapped_type = entity_type_mapping.get(entity_type)
        if not mapped_type:
            # Если нет соответствия, считаем токен не-сущностью
            new_labels.append(train_label2id['O'])
            continue
            
        # Преобразуем BIOLU -> BI
        if position in ['B', 'U']:  # Начало сущности или единичная сущность -> B
            new_label = f'B-{mapped_type}'
        elif position in ['I', 'L']:  # Внутри сущности или конец сущности -> I
            new_label = f'I-{mapped_type}'
        else:
            new_label = 'O'
            
        # Преобразуем в числовой формат
        new_labels.append(train_label2id.get(new_label, train_label2id['O']))
        
    # Обновляем пример
    example['labels'] = new_labels
    return example

- оставляем только LOC и PER, тк других похожих на наш целевой датасет сущностей нет

In [29]:
lenta_train_dataset = lenta_train_dataset.map(convert_lenta_labels_to_train_format)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
# Оставляем только 3000 примеров из датасета
lenta_train_dataset = lenta_train_dataset.select(range(3000))

In [31]:
from datasets import concatenate_datasets

# Объединение тренировочных датасетов
combined_train_dataset = concatenate_datasets([lenta_train_dataset, train_dataset])

In [33]:
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained(
    "cointegrated/rubert-tiny2", 
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir='../data/models/rubert-tiny2-ner-v3',
    eval_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    no_cuda=not torch.cuda.is_available(),
    dataloader_num_workers=8,
    dataloader_pin_memory=True,
    dataloader_prefetch_factor=2    
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=combined_train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

/var/folders/gb/r8_rgfms4txcy1_nl4l107kc0000gn/T/ipykernel_38013/3256264858.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling pa

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.457694,0.072017,0.104856,0.085388,0.847202
2,0.368800,0.393128,0.115288,0.160186,0.134078,0.860811
3,0.185100,0.365139,0.168240,0.230992,0.194684,0.871996


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

TrainOutput(global_step=1425, training_loss=0.24291555839672424, metrics={'train_runtime': 1466.7503, 'train_samples_per_second': 7.772, 'train_steps_per_second': 0.972, 'total_flos': 76468786882368.0, 'train_loss': 0.24291555839672424, 'epoch': 3.0})

- этот эксперимент менее удачный: возможно повлияло качество разметки синтетических данных или пересечение только с двумя ner классами с изначальным датасетом

- подход с дообучением на задачу mlm и последующим дообучением на задачу ner дал более качественный результат. скорее всего при mlm модель начинает лучше улавливать взаимосвязи между словами в нашем корпусе, что позволяет ей показывать хорошие результаты при извлечении именованых сущностей